# RAG Pipeline: PDF to FAISS Vector Store

This notebook demonstrates how to:
1. Load and chunk a PDF document
2. Create embeddings and store them in FAISS
3. Save the index locally as .faiss and .pkl files
4. Load the saved index and create a retriever for RAG applications

## 1. Install Required Libraries

Run this cell first to install all necessary dependencies:

## 2. Import Required Libraries

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("All libraries imported successfully!")

All libraries imported successfully!


## 3. Configuration

Set your PDF path and output directory here:

In [10]:
# Configuration
PDF_PATH = "../data/dc2523af.pdf"  # Change this to your PDF path
INDEX_SAVE_DIR = "../faiss_index"  # Directory to save the FAISS index
EMBEDDING_MODEL = "text-embedding-3-large"  # Embedding model to use

# Text splitting parameters
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

print(f"PDF Path: {PDF_PATH}")
print(f"Index Save Directory: {INDEX_SAVE_DIR}")
print(f"Embedding Model: {EMBEDDING_MODEL}")

PDF Path: ../data/dc2523af.pdf
Index Save Directory: ../faiss_index
Embedding Model: text-embedding-3-large


In [11]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
model_name = "gpt-4o"
llm = ChatOpenAI(
                model=model_name,
                #openai_api_base=openai_api_base
            )

## 4. Initialize Components

In [12]:
# Initialize embeddings
print("Loading embedding model...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

print("Components initialized successfully!")

Loading embedding model...
Components initialized successfully!


## 5. Load and Process PDF

In [13]:
# Load PDF
print(f"Loading PDF from {PDF_PATH}...")
loader = PyPDFLoader(PDF_PATH)
pages = loader.load()

print(f"Loaded {len(pages)} pages from PDF")
print(f"First page preview (first 200 chars): {pages[0].page_content[:200]}...")

Loading PDF from ../data/dc2523af.pdf...
Loaded 69 pages from PDF
First page preview (first 200 chars): ...


## 6. Split Documents into Chunks

In [14]:
# Split documents into chunks
print("Splitting documents into chunks...")
chunks = text_splitter.split_documents(pages)

print(f"Created {len(chunks)} chunks")
print(f"First chunk preview:\n{chunks[0].page_content[:200]}...")
print(f"Average chunk length: {sum(len(chunk.page_content) for chunk in chunks) // len(chunks)} characters")

Splitting documents into chunks...
Created 130 chunks
First chunk preview:
1 Introduction ........................................................................................  5
1.1 Ownership of this document 5
1.2	 API	Definition	and	Overview	 5
1.3 Purpose 6
1.4 Scope ...
Average chunk length: 767 characters


## 7. Create FAISS Index

In [15]:
# Create FAISS vectorstore
print("Creating FAISS index")
vectorstore = FAISS.from_documents(chunks, embeddings)

print("FAISS index created successfully!")
print(f"Index contains {vectorstore.index.ntotal} vectors")

Creating FAISS index


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
INFO:faiss:Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.


FAISS index created successfully!
Index contains 130 vectors


## 8. Save Index Locally

In [17]:
# Create directory if it doesn't exist
os.makedirs(INDEX_SAVE_DIR, exist_ok=True)

# Save FAISS index locally
print(f"Saving FAISS index to {INDEX_SAVE_DIR}...")
vectorstore.save_local(INDEX_SAVE_DIR)

print("Index saved successfully!")

# List files in the directory
files = os.listdir(INDEX_SAVE_DIR)
print(f"Files created: {files}")

Saving FAISS index to ../faiss_index...
Index saved successfully!
Files created: ['index.faiss', 'index.pkl']


## 9. Load Saved Index

In [18]:
# Load the saved FAISS index
print(f"Loading FAISS index from {INDEX_SAVE_DIR}...")
loaded_vectorstore = FAISS.load_local(
    INDEX_SAVE_DIR, 
    embeddings,
    allow_dangerous_deserialization=True
)

print("Index loaded successfully!")
print(f"Loaded index contains {loaded_vectorstore.index.ntotal} vectors")

Loading FAISS index from ../faiss_index...
Index loaded successfully!
Loaded index contains 130 vectors


## 10. Create Retriever for RAG

In [19]:
# Create retriever
retriever = loaded_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # Number of documents to retrieve
)

print("Retriever created successfully!")
print(f"Retriever configured to return top 4 most similar documents")

Retriever created successfully!
Retriever configured to return top 4 most similar documents


## 11. Test the Retriever

In [14]:
# Test the retriever with a sample query
test_query = "What is the main topic of this document?"  # Change this to test with your content

print(f"Testing retriever with query: '{test_query}'")
results = retriever.get_relevant_documents(test_query)

print(f"Retrieved {len(results)} documents")
print("\n" + "="*50)
for i, doc in enumerate(results):
    print(f"\nDocument {i+1}:")
    print(f"Content: {doc.page_content[:300]}...")
    print(f"Metadata: {doc.metadata}")
    print("-" * 30)

Testing retriever with query: 'What is the main topic of this document?'


/var/folders/kh/khjtwkx55nz26qv154hsd08h0000gn/T/ipykernel_12962/3480392225.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(test_query)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Retrieved 4 documents


Document 1:
Content: contain	no	explicit	technology	or	protocol	restrictions;	rather,	the	document	offers	best	practices-
based	guidelines	that	ensure	that	UAE	digital	government	APIs	are	effective,	designed	correctly,	
secure	and	provide	value.
1.3.	Purpose...
Metadata: {'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.0 (Macintosh)', 'creationdate': '2021-04-29T13:43:36+04:00', 'moddate': '2021-05-02T11:10:50+04:00', 'trapped': '/False', 'source': '../data/dc2523af.pdf', 'total_pages': 69, 'page': 7, 'page_label': '7'}
------------------------------

Document 2:
Content: UAE Government API First Guidlines
4
1.
Ownership
of this document
Introduction
1.1....
Metadata: {'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.0 (Macintosh)', 'creationdate': '2021-04-29T13:43:36+04:00', 'moddate': '2021-05-02T11:10:50+04:00', 'trapped': '/False', 'source': '../data/dc2523af.pdf', 'total_pages': 69, 'page': 4, 'page_label': '4'}
-----

## 12. Utility Functions for Reusability

In [20]:
def load_or_create_vectorstore(pdf_path: str, index_dir: str, force_recreate: bool = False):
    """
    Load existing vectorstore or create new one if it doesn't exist.
    
    Args:
        pdf_path: Path to PDF file
        index_dir: Directory for FAISS index
        force_recreate: Whether to recreate index even if it exists
    
    Returns:
        FAISS vectorstore object
    """
    if not force_recreate and os.path.exists(index_dir) and os.listdir(index_dir):
        print(f"Loading existing index from {index_dir}")
        return FAISS.load_local(
            index_dir, 
            embeddings, 
            allow_dangerous_deserialization=True
        )
    else:
        print(f"Creating new index from {pdf_path}")
        # Load and process PDF
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()
        chunks = text_splitter.split_documents(pages)
        
        # Create and save vectorstore
        vectorstore = FAISS.from_documents(chunks, embeddings)
        os.makedirs(index_dir, exist_ok=True)
        vectorstore.save_local(index_dir)
        
        return vectorstore

def create_retriever(pdf_path: str, index_dir: str, k: int = 4, force_recreate: bool = False):
    """
    Create a retriever from PDF.
    
    Args:
        pdf_path: Path to PDF file
        index_dir: Directory for FAISS index
        k: Number of documents to retrieve
        force_recreate: Whether to recreate index
    
    Returns:
        LangChain retriever object
    """
    vectorstore = load_or_create_vectorstore(pdf_path, index_dir, force_recreate)
    return vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": k}
    )

print("Utility functions defined!")

Utility functions defined!


## 13. Example: Using the Utility Functions

In [21]:
# Example of using utility functions
# This will load existing index or create new one if needed
example_retriever = create_retriever(
    pdf_path=PDF_PATH,
    index_dir=INDEX_SAVE_DIR,
    k=3,  # Retrieve top 3 documents
    force_recreate=False  # Set to True to force recreation
)

print("Example retriever created!")

# Test with a different query
test_query_2 = "main concepts"  # Change this for your specific content
results_2 = example_retriever.get_relevant_documents(test_query_2)

print(f"\n Query: '{test_query_2}'")
print(f"Found {len(results_2)} relevant documents")
for i, doc in enumerate(results_2):
    print(f"\nDoc {i+1}: {doc.page_content[:150]}...")

Loading existing index from ../faiss_index
Example retriever created!


/var/folders/kh/khjtwkx55nz26qv154hsd08h0000gn/T/ipykernel_36565/1846000481.py:14: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results_2 = example_retriever.get_relevant_documents(test_query_2)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"



 Query: 'main concepts'
Found 3 relevant documents

Doc 1: standards	 supported.
8.	Loosely	coupled	–	provide	flexibility	and	minimize	impact	of	
changes	to	operations	of	other	APIs.
9.	Granularity	–	provide	t...

Doc 2: early	rather	than	too	late)	and	frequent	delivery	of	products.
•  
Configuration	Management	-	All	the	components	which	make	
up	an	instance	of	the	API...

Doc 3: UAE Government API First Guidlines
14
This	section	of	the	document	considers	the	business	and	operational	
context	 for	 APIs	 within	 the	 UAE	 gover...


## 14. Advanced RAG Chain with Question Rewriting

Now let's create a more sophisticated RAG chain that includes:
- Question rewriting based on chat history
- Context-aware retrieval
- Conversational response generation

In [6]:
# Additional imports for advanced RAG chain
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from operator import itemgetter
from typing import Dict, Any

print("Additional imports loaded!")

Additional imports loaded!


## 16. Define Prompts for RAG Chain

In [36]:
# Contextualize prompt for question rewriting
contextualize_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is.

CRITICAL: You MUST respond in {language}. This is mandatory - never use English unless language is "English".
"""

contextualize_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# QA prompt for final answer generation
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.

CRITICAL INSTRUCTION: You MUST respond in {language}. Do not use English unless language is specifically "English". This is mandatory.

Context: {context}
"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

print("Prompts defined!")

Prompts defined!


## 17. Create RAG Chain Class

In [37]:
class AdvancedRAGChain:
    def __init__(self, retriever, llm, contextualize_prompt, qa_prompt):
        self.retriever = retriever
        self.llm = llm
        self.contextualize_prompt = contextualize_prompt
        self.qa_prompt = qa_prompt
        
        # Build the chain
        self._build_chain()
    
    def _format_docs(self, docs):
        """Format retrieved documents into a single string."""
        return "\n\n".join(doc.page_content for doc in docs)
    
    def _build_chain(self):
        """Build the complete RAG chain."""
        # 1) Question rewriter based on chat history
        question_rewriter = (
            {"input": itemgetter("input"), "chat_history": itemgetter("chat_history"), "language":itemgetter("language")}
            | self.contextualize_prompt
            | self.llm
            | StrOutputParser()
        )
        
        # 2) Retrieve docs for rewritten question
        retrieve_docs = question_rewriter | self.retriever | self._format_docs
        
        # 3) Answer using retrieved context + original input + chat history
        self.chain = (
            {
                "context": retrieve_docs,
                "input": itemgetter("input"),
                "chat_history": itemgetter("chat_history"),
                "language":itemgetter("language"),
            }
            | self.qa_prompt
            | self.llm
            | StrOutputParser()
        )
    
    def invoke(self, input_dict: Dict[str, Any]) -> str:
        """Invoke the RAG chain."""
        return self.chain.invoke(input_dict)
    
    def stream(self, input_dict: Dict[str, Any]):
        """Stream the RAG chain response."""
        return self.chain.stream(input_dict)

print("AdvancedRAGChain class defined!")

AdvancedRAGChain class defined!


## 18. Initialize the Advanced RAG Chain

In [38]:
# Create the advanced RAG chain using our existing retriever
rag_chain = AdvancedRAGChain(
    retriever=retriever,
    llm=llm,
    contextualize_prompt=contextualize_prompt,
    qa_prompt=qa_prompt
)

print("Advanced RAG Chain initialized!")

Advanced RAG Chain initialized!


In [43]:
response = rag_chain.invoke({
    "input": "what principle to follow when create an API?",  # Ask in French
    "chat_history": [],
    "language": "Arabic"
})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [44]:
response

'عند إنشاء واجهة برمجة تطبيقات (API)، يجب اتباع عدة مبادئ أساسية، منها: القابلية للاستخدام لضمان تجربة مستخدم عالية الجودة، التوافقية لتمكين تبادل البيانات بدون تبعيات على التقنيات الأساسية، وإعادة الاستخدام لتجنب التكرار في الجهود. بالإضافة إلى الاستقلالية لتجنب الاعتماد على مزودين أو تقنيات معينة، وقابلية التوسع لتوفير المرونة، والاستقرار لضمان التناسق في التغييرات، وكذلك الشفافية والاتصال المحدود لتقليل تأثير التغييرات على العمليات الأخرى.'

## 19. Test the Advanced RAG Chain

In [35]:
# Test the advanced RAG chain with conversation
from langchain.schema import HumanMessage, AIMessage

# Test 1: First question (no chat history)

response1 = rag_chain.invoke({
    "input": "What is the main topic of this document?",
    "chat_history": [],
    "language":"French"
})
print(f"Response: {response1}")
print("\n" + "="*50 + "\n")

# Test 2: Follow-up question with chat history
chat_history = [
    HumanMessage(content="What is the main topic of this document?"),
    AIMessage(content=response1)
]

response2 = rag_chain.invoke({
    "input": "Can you tell me more about it?",  # This references previous context
    "chat_history": chat_history,
    "language":"French"
})
print(f"Response: {response2}")
print("\n" + "="*50 + "\n")

# Test 3: New specific question
print("🔍 Test 3: Specific question")
response3 = rag_chain.invoke({
    "input": "What are the key points mentioned?",
    "chat_history": chat_history,
    "language":"French"
})
print(f"Response: {response3}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: The main topic of this document is the guidelines and best practices for the effective design, security, and value provision of UAE digital government APIs. It covers various aspects such as API lifecycle management, documentation generation, and business and operational contexts within the UAE government. Additionally, it discusses principles that impact API creation and their broader effects across government entities and public services.




INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: This document serves as a tool for planning and implementing digital transformation in UAE government entities through the use of APIs. It aims to make government services more interoperable, less dependent on specific vendors or technologies, and easier to update. It provides high-level guidelines for API design and implementation, along with data and information security standards to ensure consistent communication and secure information exchange across government services.


🔍 Test 3: Specific question


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: The key points mentioned include ensuring that government services are interoperable, vendor-agnostic, and future-proof with guidelines for API design that promote a uniform design language. The document emphasizes the importance of consumer-centric design, considering the needs of API consumers and developers, to create intuitive and easy-to-use APIs. It also highlights the significance of data standards and information security standards to maintain secure and consistent data exchange across government services.


## 20. Conversational RAG Helper Function

In [27]:
class ConversationalRAG:
    def __init__(self, rag_chain):
        self.rag_chain = rag_chain
        self.chat_history = []
    
    def ask(self, question: str) -> str:
        """Ask a question and maintain chat history."""
        response = self.rag_chain.invoke({
            "input": question,
            "chat_history": self.chat_history
        })
        
        # Update chat history
        self.chat_history.append(HumanMessage(content=question))
        self.chat_history.append(AIMessage(content=response))
        
        return response
    
    def clear_history(self):
        """Clear the chat history."""
        self.chat_history = []
        print("Chat history cleared!")
    
    def get_history(self):
        """Get the current chat history."""
        return self.chat_history

# Create conversational RAG instance
conv_rag = ConversationalRAG(rag_chain)

print("Conversational RAG helper created!")

Conversational RAG helper created!


## 21. Interactive Conversation Example

In [22]:
# Example conversation
print("Starting interactive conversation...\n")

# Question 1
q1 = "What are the main concepts in this document?"
print(f"Human: {q1}")
a1 = conv_rag.ask(q1)
print(f"Assistant: {a1}\n")

# Question 2 (references previous context)
q2 = "Can you elaborate on the first concept you mentioned?"
print(f"Human: {q2}")
a2 = conv_rag.ask(q2)
print(f"Assistant: {a2}\n")

# Question 3
q3 = "Are there any examples provided?"
print(f"Human: {q3}")
a3 = conv_rag.ask(q3)
print(f"Assistant: {a3}\n")

print(f"Conversation history contains {len(conv_rag.get_history())} messages")

Starting interactive conversation...

Human: What are the main concepts in this document?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant: The main concepts in this document are API-first guidelines for UAE government entities, focusing on accelerating digital transformation through best practices in API design and development. It emphasizes the importance of rate limiting, versioning information, and the impact of APIs on government and public services. The document serves as a comprehensive guide to ensure that UAE digital government APIs are effective, innovative, and customer-focused.

Human: Can you elaborate on the first concept you mentioned?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant: The document emphasizes API-first guidelines for UAE government entities as a critical concept, highlighting their role in driving digital transformation. It provides high-level guidelines and low-level best practices to ensure that APIs are seamlessly integrated, interoperable with other platforms, and not tied to specific vendors or technologies. This approach aims to increase the efficiency, interoperability, and future-proofing of government services, while also facilitating real-time processing and flexibility in updating and testing APIs.

Human: Are there any examples provided?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant: The document does not provide specific examples but focuses on overarching concepts and guidelines for designing and implementing APIs within UAE government entities. It outlines the goals and benefits of adopting an API-first strategy, such as enhancing interoperability, reducing vendor lock-in, and ensuring services are more future-proof and easier to update.

Conversation history contains 6 messages


In [23]:
conv_rag.get_history()


[HumanMessage(content='What are the main concepts in this document?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The main concepts in this document are API-first guidelines for UAE government entities, focusing on accelerating digital transformation through best practices in API design and development. It emphasizes the importance of rate limiting, versioning information, and the impact of APIs on government and public services. The document serves as a comprehensive guide to ensure that UAE digital government APIs are effective, innovative, and customer-focused.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Can you elaborate on the first concept you mentioned?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The document emphasizes API-first guidelines for UAE government entities as a critical concept, highlighting their role in driving digital transformation. It provides high-level guidelines and low-level best practices

In [1]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
chain = RunnablePassthrough() | RunnablePassthrough()
chain.invoke("hey there")

'hey there'

In [3]:
def str_upper(input):
    return input.upper()
chain = RunnablePassthrough() | RunnableLambda(str_upper)
chain.invoke("hey there")

'HEY THERE'

## 22. Summary

🎉 **Congratulations!** You have successfully created a complete advanced RAG system with:

### Core Features:
1. ✅ **PDF Processing**: Loaded and chunked PDF documents
2. ✅ **Vector Storage**: Created and saved FAISS index locally
3. ✅ **Question Rewriting**: Context-aware question reformulation
4. ✅ **Conversational Memory**: Maintains chat history for follow-up questions
5. ✅ **Advanced Retrieval**: Retrieves relevant documents based on rewritten questions
6. ✅ **Response Generation**: Generates contextual answers using retrieved documents

### Key Components:
- **Question Rewriter**: Reformulates questions based on chat history
- **Document Retriever**: Finds relevant chunks from your PDF
- **Response Generator**: Creates answers using retrieved context
- **Conversational Interface**: Maintains conversation flow

### Files Created:
- `index.faiss`: The FAISS index file
- `index.pkl`: Metadata and document store

### Usage:
```python
# Simple usage
response = rag_chain.invoke({
    "input": "Your question here",
    "chat_history": []
})

# Conversational usage
conv_rag = ConversationalRAG(rag_chain)
answer = conv_rag.ask("Your question here")
```

### Next Steps:
- Replace the demonstration LLM with your preferred model (OpenAI, Claude, etc.)
- Customize prompts for your specific use case
- Add multiple PDF documents to expand the knowledge base
- Implement streaming responses for better user experience
- Add evaluation metrics to measure RAG performance